# Digital Image Processing Coursework
Danupat Khamnuansin
### Slide is here

# HW1: Greyscale

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a **grayscale** or **greyscale** digital image is an image in which the value of each pixel is a single sample, that is, it carries only intensity information. Images of this sort, also known as black-and-white, are composed exclusively of shades of gray, varying from black at the weakest intensity to white at the strongest.

## Formula
- Gray = (Red + Green + Blue) / 3
- Gray = (Red * 0.3 + Green * 0.59 + Blue * 0.11) >>>>>> Photoshop used
- Gray = (Red * 0.2126 + Green * 0.7152 + Blue * 0.0722) >>>>>> called "LUMA" (BT.709)
- Gray = (Red * 0.299 + Green * 0.587 + Blue * 0.114) >>>>>> BT.601

### Readmore
http://www.tannerhelland.com/3643/grayscale-image-algorithm-vb6/

### Hard-code

In [81]:
import sys
from PIL import Image

def main(name):
    input_img = Image.open(name).convert('RGB')
    w, h = input_img.size
    for x in range(w):
        for y in range(h):
            current = input_img.getpixel((x, y))
            grey = sum(current)//3 # Change formular here
            input_img.putpixel((x, y), (grey,grey,grey))
    # Save image
    input_img.save('1_greyscale/out_hard.png')
if __name__ == '__main__':
    try:
        main(sys.argv[-1])
    except:
        main('1_greyscale/in.png')

### Library-use

In [82]:
import sys
from PIL import Image

def main(name):
    img = Image.open(name).convert('LA')
    img.save('1_greyscale/out_lib.png')

if __name__ == '__main__':
    try:
        main(sys.argv[-1])
    except:
        main('1_greyscale/in.png')

## Input
<img src="1_greyscale/in.png" alt="Drawing" style="width: 500px;"/> <br>
## Output
<img src="1_greyscale/out_hard.png" alt="Drawing" style="width: 500px;"/>
<center><b>Hard-code</b></center>
<br>
<img src="1_greyscale/out_lib.png" alt="Drawing" style="width: 500px;"/>
<center><b>Library-use</b></center>

# HW2: Find Connected Component

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Flood fill**, also called **seed fill**, is an algorithm that determines the area connected to a given node in a multi-dimensional array. It is used in the "bucket" fill tool of paint programs to fill connected, similarly-colored areas with a different color, and in games such as Go and Minesweeper for determining which pieces are cleared. When applied on an image to fill a particular bounded area with color, it is also known as boundary fill.

###  Hard-code

In [3]:
from PIL import Image
import random as rd
from collections import deque
import sys

WHITE = 255, 255, 255
BLACK = 0, 0, 0

def gen_colors(n):
    colors = []
    i = 0
    while i < n:
        color = (rd.randrange(255),rd.randrange(255),rd.randrange(255))
        if color != BLACK and color != WHITE:
            colors.append(color)
            i += 1
    return colors

def main(name):
    input_img = Image.open(name).convert('RGB')
    w, h = input_img.size
    colors = gen_colors(100)
    i = 0
    for x in range(w):
        for y in range(h):
            current = input_img.getpixel((x, y))
            if current not in colors:
                flood_fill(x, y, colors[i], input_img, w, h)
                if i + 1 >= len(colors):
                    break
                i += 1
    # Save image
    input_img.save('2_concom/out.png')

def flood_fill(x, y, color, image, w, h):
    source = image.getpixel((x, y))
    if source != color:
        pixels = deque([(x, y)])
        while pixels:
            x, y = place = pixels.popleft()
            image.putpixel(place, color)
            for x_offset in -1,0,1:
                x_offset += x
                for y_offset in -1,0,1:
                    y_offset += y
                    new_place = x_offset, y_offset
                    if x_offset < 0 or x_offset >= w or y_offset < 0 or y_offset >= h:
                        continue
                    if image.getpixel(new_place) == source and new_place not in pixels:
                        pixels.append(new_place)

if __name__ == '__main__':
    try:
        main(sys.argv[-1])
    except:
        main('2_concom/in.png')

## Input
<img src="2_concom/in.png" alt="Drawing" style="width: 500px;"/> <br> 
## Output
<img src="2_concom/out.png" alt="Drawing" style="width: 500px;"/>

# HW3: Histogram Equalization
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; This method usually increases the global contrast of many images, especially when the usable data of the image is represented by close contrast values. Through this adjustment, the intensities can be better distributed on the histogram. This allows for areas of lower local contrast to gain a higher contrast. Histogram equalization accomplishes this by effectively spreading out the most frequent intensity values.

###  Hard-code

In [79]:
from PIL import Image
import numpy as np
import math

# Algorithm
def equalize_hist(img):
    data = img.copy().flatten()
    histogram = [0]*256
    for pixel in data:
        histogram[pixel] += 1
    cdf = [0]*256
    for i in range (len(histogram)):
        cdf[i] = sum(histogram[0:i+1])
    
    prob_of_cdf = [0]*256
    for i in range (len(cdf)):
        prob_of_cdf[i] = cdf[i]/cdf[-1]
    
    new_map = [0]*256
    for i in range (len(new_map)):
        new_map[i] = math.floor(prob_of_cdf[i]*255)

    new_img = np.array([[0]*len(img[0])]*len(img), dtype=np.uint8)
    for x in range (len(img)):
        for y in range (len(img[0])):
            new_img[x][y] = new_map[img[x][y]]
    return new_img

if __name__ == '__main__':
    img = Image.open('3_histeq/in.png')
    img.load()
    data = np.asarray( img, dtype=np.uint8 )
    img_eq = Image.fromarray(equalize_hist(data), mode='L')
    img_eq.save('3_histeq/out_hard.png')
    

### Library-use

In [80]:
from skimage import exposure
import sys
from PIL import Image
import numpy as np

def main(name):
    img = Image.open(name)
    img.load()
    data = np.asarray( img, dtype="int32" )
    img_eq_array = exposure.equalize_hist(data)
    img_eq = Image.fromarray(img_eq_array)
    img_eq.save('3_histeq/out_lib.tiff')

if __name__ == '__main__':
    main('3_histeq/in.png')

## Input
<img src="3_histeq/in.png" alt="Drawing" style="width: 500px;"/> <br> 
## Output
<img src="3_histeq/out_hard.png" alt="Drawing" style="width: 500px;"/>
<center><b>Hard-code</b></center>
<br>
<img src="3_histeq/out_lib.tiff" alt="Drawing" style="width: 500px;"/>
<center><b>Library-use</b></center>